In [ ]:
import time
import numpy as np
import pandas as pd
import scipy
import math
import random as rand
import pickle

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib import cm
from matplotlib import animation as animation
import matplotlib as mpl

from PIL import Image
from pylab import *

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import train_test_split

from skimage.metrics import structural_similarity as ssim
from scipy.interpolate import griddata

from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Input, Conv2D, Concatenate
from keras.models import Model
import tensorflow as tf

from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

In [ ]:
vor_train = np.load('/content/drive/MyDrive/Physics/Physics/Dataset10/vor_train.npy', mmap_mode="r")[:30*300]
true_train = np.load('/content/drive/MyDrive/Physics/Physics/Dataset10/true_train.npy', mmap_mode="r")[:30*300]
vor_test = np.load('/content/drive/MyDrive/Physics/Physics/Dataset10/vor_test.npy', mmap_mode="r")
true_test = np.load('/content/drive/MyDrive/Physics/Physics/Dataset10/true_test.npy', mmap_mode="r")

# DSOVT - CED-LSTM - Basic Version

## CED 

In [ ]:
tf.keras.utils.set_random_seed(1)
def custom_activation(x):
    tanh_activation = tf.keras.activations.tanh(x[:, :, :, :2])
    scaled_channel_3 = tf.keras.activations.relu(x[:, :, :, 2])
    return tf.concat([tanh_activation, tf.expand_dims(scaled_channel_3, axis=-1)], axis=-1)
# Encoder
input_img = layers.Input(shape=(64, 64, 3))
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.Flatten()(x)
encoded = layers.Dense(128, activation='relu', name='encoded')(x)

# Decoder
x = layers.Dense(16*16*64, activation='relu')(encoded)
x = layers.Reshape((16, 16, 64))(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2D(3, (3, 3), activation=custom_activation, padding='same', name='decoded')(x)

# Autoencoder Model
autoencoder = models.Model(input_img, decoded)


autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.summary()

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

# Set the random seed for reproducibility
tf.keras.utils.set_random_seed(42)

# Callbacks for adaptive learning rate, early stopping, and model checkpoint
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    patience=55,
    min_lr=1e-5
)
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=60,
    restore_best_weights=True
)
model_checkpoint = ModelCheckpoint(
    './Model_para/CED_LSTM/CED-Direct-SW-image',#notry
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

history = autoencoder.fit(
    vor_train,
    true_train,
    epochs=200,
    batch_size=8,
    callbacks=[reduce_lr, early_stop, model_checkpoint],
    validation_data=(vor_test, true_test),
    shuffle=True
)

In [ ]:
vor_image = np.vstack([vor_train,vor_test])
true_image = np.vstack([true_train,true_test])

In [ ]:
encoder = tf.keras.models.Model(input_img, encoded)

encoded_features = encoder.predict(vor_image)
reconstructed_images = autoencoder.predict(vor_image)

In [ ]:
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
import numpy as np

def calculate_rrmse(true, predicted):
    """
    Calculate the Relative Root Mean Square Error (R-RMSE).
    """
    # Calculate the Root Mean Square Error (RMSE)
    rmse = np.sqrt(np.mean((true - predicted) ** 2))

    # Calculate the mean of the true data
    mean_true = np.mean(true)

    # Avoid division by zero by handling the case where the mean of the true data is zero
    if mean_true == 0:
        return np.inf

    # Calculate the Relative Root Mean Square Error (R-RMSE)
    rrmse = rmse / mean_true
    return rrmse

# Initialize SSIM and PSNR lists
ssim_values = []
psnr_values = []

# Calculate SSIM and PSNR for each pair of images
for i in range(len(true_test)):
    ssim_val = ssim(true_test[i], reconstructed_images[i], multichannel=True)
    # Use a consistent data_range to calculate PSNR
    psnr_val = psnr(true_test[i], reconstructed_images[i], data_range=2)

    ssim_values.append(ssim_val)
    psnr_values.append(psnr_val)

# Calculate average SSIM and PSNR values
average_ssim = np.mean(ssim_values)
average_psnr = np.mean(psnr_values)

print(f"Average SSIM: {average_ssim}")
print(f"Average PSNR: {average_psnr}")
print(f"Average R-RMSE: {calculate_rrmse(true_test, reconstructed_images)}")


## LSTM

In [ ]:
# Split encoded features into datasets
datasets = np.split(encoded_features, 40)

# Split datasets into training and testing indices
train_indices, test_indices = train_test_split(np.arange(40), test_size=1/4, shuffle=False)

# Function to prepare training and validation data
def prepare_data(indices, datasets, input_length=5, forecast_horizon=5):
    X, y = [], []
    for idx in indices:
        data = datasets[idx]

        # Generate input-output pairs
        for i in range(len(data) - input_length - forecast_horizon + 1):
            X.append(data[i:(i + input_length)])
            y.append(data[(i + input_length):(i + input_length + forecast_horizon)])
    return np.array(X), np.array(y)

# Function to prepare testing data
def prepare_data_image(indices, true_image, input_length=5, forecast_horizon=5):
    y = []
    for idx in indices:
        data = true_image[idx]

        # Generate output sequences
        for i in range(len(data) - input_length - forecast_horizon + 1):
            y.append(data[(i + input_length):(i + input_length + forecast_horizon)])
    return np.array(y)

# Prepare training and validation data
x_train, y_train = prepare_data(train_indices, datasets)
x_val, y_val = prepare_data(test_indices, datasets)

# Prepare testing data for images
y_test_image = prepare_data_image(test_indices, true_image.reshape(40, 300, 64, 64, 3))


In [ ]:
class OriginalModel(tf.keras.Model):
    def __init__(self, n_step=5, feature_dim=128):
        super(OriginalModel, self).__init__()
        self.n_step = n_step
        self.feature_dim = feature_dim
        # Define the model layers
        self.lstm1 = tf.keras.layers.LSTM(32, return_sequences=False)
        self.repeatVector = tf.keras.layers.RepeatVector(n_step)
        self.lstm2 = tf.keras.layers.LSTM(32, return_sequences=True)
        self.timeDistributed = tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(feature_dim))
        self.activation = tf.keras.layers.Activation('relu')

    def call(self, inputs):
        x = self.lstm1(inputs)
        x = self.repeatVector(x)
        x = self.lstm2(x)
        x = self.timeDistributed(x)
        return self.activation(x)

    def train_step(self, data):
        x_true, y_true = data  # Unpack the data

        with tf.GradientTape() as tape:
            y_pred = self(x_true, training=True)  # Forward pass
            loss = tf.keras.losses.mean_squared_error(y_true, y_pred)  # Compute loss

        gradients = tape.gradient(loss, self.trainable_variables)  # Compute gradients
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))  # Update model parameters

        # Update metrics
        self.compiled_metrics.update_state(y_true, y_pred)
        results = {m.name: m.result() for m in self.metrics}
        return results

    def test_step(self, data):
        x_true, y_true = data
        y_pred = self(x_true, training=False)
        loss = tf.keras.losses.mean_squared_error(y_true, y_pred)

        # Update metrics
        self.compiled_metrics.update_state(y_true, y_pred)
        results = {m.name: m.result() for m in self.metrics}
        return results

In [ ]:
# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Define early stopping callback
early_stopping = EarlyStopping(
    monitor='val_mean_squared_error',
    patience=15,
    verbose=1,
    mode='min',
    restore_best_weights=True
)

# Define model checkpoint callback
model_checkpoint = ModelCheckpoint(
    filepath='./Model_para/CED_LSTM/LSTM-Basic-SW',
    monitor='val_mean_squared_error',
    save_best_only=True,
    verbose=1,
    mode='min'
)

# Define reduce learning rate on plateau callback
reduce_lr = ReduceLROnPlateau(
    monitor='val_mean_squared_error',
    factor=0.5,
    patience=20,
    verbose=1,
    mode='min',
    min_lr=1e-4
)

# Set random seed again for TensorFlow's internal random number generators
tf.keras.utils.set_random_seed(1)

# Create an instance of OriginalModel
Orig_model = OriginalModel(n_step=5, feature_dim=128)

# Compile the model
Orig_model.compile(optimizer=tf.keras.optimizers.Adam(0.01), metrics=['mean_squared_error'])

# Train the model
Orig_model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=150, batch_size=16, callbacks=[model_checkpoint, reduce_lr])

In [ ]:
# Record the starting time
start_time = time.time()

# Reshape the encoded vector to match the input shape expected by the decoder
encoded_vector = Orig_model.predict(x_val)[:, 0, :].reshape((-1, 128))  # Assuming encoded_vector is a 1D numpy array

# Decode the encoded vector using the decoder model
decoded_image = decoder.predict(encoded_vector)

# Calculate the total inference time
inference_time = time.time() - start_time
print("Inference time:", inference_time)

In [ ]:
true_images = y_test_image[:, 0, :, :, :]
reconstructed_images = decoded_image

def calculate_rrmse(true, predicted):
    """
    Calculate the Relative Root Mean Square Error (R-RMSE).
    """
    # Calculate the Root Mean Square Error (RMSE)
    rmse = np.sqrt(np.mean((true - predicted) ** 2))

    # Calculate the mean of the true data
    mean_true = np.mean(true)

    # Avoid division by zero by handling the case where the mean of the true data is zero
    if mean_true == 0:
        return np.inf

    # Calculate the Relative Root Mean Square Error (R-RMSE)
    rrmse = rmse / mean_true
    return rrmse

# Initialize SSIM and PSNR lists
ssim_values = []
psnr_values = []

# Calculate SSIM and PSNR for each pair of images
for i in range(len(true_images)):
    ssim_val = ssim(true_images[i], reconstructed_images[i], multichannel=True)
    # Use a consistent data_range to calculate PSNR
    psnr_val = psnr(true_images[i], reconstructed_images[i], data_range=2)

    ssim_values.append(ssim_val)
    psnr_values.append(psnr_val)

# Calculate average SSIM and PSNR values
average_ssim = np.mean(ssim_values)
average_psnr = np.mean(psnr_values)

print(f"Average SSIM: {average_ssim}")
print(f"Average PSNR: {average_psnr}")
print(f"Average R-RMSE: {calculate_rrmse(true_images, reconstructed_images)}")


# DSOVT - CED-LSTM - Physics Contrained Version

In [ ]:
vor_train = np.load('/content/drive/MyDrive/Physics/Physics/Dataset10/vor_train.npy', mmap_mode="r")[:20*300]
true_train = np.load('/content/drive/MyDrive/Physics/Physics/Dataset10/true_train.npy', mmap_mode="r")[:20*300]
vor_test = np.load('/content/drive/MyDrive/Physics/Physics/Dataset10/vor_test.npy', mmap_mode="r")
true_test = np.load('/content/drive/MyDrive/Physics/Physics/Dataset10/true_test.npy', mmap_mode="r")

## CED

In [ ]:
tf.keras.utils.set_random_seed(1)
def custom_activation(x):
    tanh_activation = tf.keras.activations.tanh(x[:, :, :, :2])
    scaled_channel_3 = tf.keras.activations.relu(x[:, :, :, 2])
    return tf.concat([tanh_activation, tf.expand_dims(scaled_channel_3, axis=-1)], axis=-1)
# Encoder
input_img = layers.Input(shape=(64, 64, 3))
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.Flatten()(x)
encoded = layers.Dense(64, activation='relu', name='encoded')(x)

# Decoder
x = layers.Dense(16*16*64, activation='relu')(encoded)
x = layers.Reshape((16, 16, 64))(x)
x = layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
x = layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = layers.UpSampling2D((2, 2))(x)
decoded = layers.Conv2D(3, (3, 3), activation=custom_activation, padding='same', name='decoded')(x)

# Autoencoder Model
autoencoder = models.Model(input_img, decoded)


autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.summary()

In [ ]:
tf.keras.utils.set_random_seed(42)

# Callbacks for adaptive learning rate and early stopping
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    patience=55,
    min_lr=1e-5
)
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=60,
    restore_best_weights=True
)

# Training the autoencoder
# Make sure autoencoder, vor_train, true_train, vor_test, and true_test are properly defined
history = autoencoder.fit(
    vor_train,
    true_train,
    epochs=100,
    batch_size=64,
    callbacks=[reduce_lr, early_stop],
    validation_data=(vor_test, true_test),
    shuffle=True
)

encoder = tf.keras.models.Model(input_img, encoded)

encoded_features = encoder.predict(vor_image)
reconstructed_images = autoencoder.predict(vor_image)

last_layers = autoencoder.layers[-7:]
new_model = tf.keras.models.Sequential(last_layers)

In [ ]:
model_json = autoencoder.to_json()
with open("./Model_para/CED_LSTM/SW-Phy/autoencoder_architecture.json", "w") as json_file:
    json_file.write(model_json)

autoencoder.save_weights("./Model_para/CED_LSTM/SW-Phy/autoencoder_weights.h5")
print("Saved model to disk 😄")

with open("./Model_para/CED_LSTM/SW-Phy/model_info.txt", "w") as f:
    f.write("Training Parameters and Performance Metrics\n")
    f.write("=================================================\n")
    f.write("ReduceLROnPlateau settings:\n")
    f.write(f"- Monitor: val_loss\n")
    f.write(f"- Factor: 0.2\n")
    f.write(f"- Patience: 30\n")
    f.write(f"- Min LR: 1e-6\n\n")
    f.write("EarlyStopping settings:\n")
    f.write(f"- Monitor: val_loss\n")
    f.write(f"- Patience: 40\n")
    f.write(f"- Restore Best Weights: True\n\n")
    f.write("Training History:\n")
    for key, values in history.history.items():
        f.write(f"- {key}: {values}\n")

print("All model information has been saved successfully 😄")

In [ ]:
# Combine training and testing data
vor_image = np.vstack([vor_train, vor_test])
true_image = np.vstack([true_train, true_test])

# Set random seed for reproducibility
tf.keras.utils.set_random_seed(1)

def custom_activation(x):
    """
    Custom activation function.
    Applies tanh activation to the first two channels and ReLU activation to the third channel.
    """
    tanh_activation = tf.keras.activations.tanh(x[:, :, :, :2])
    scaled_channel_3 = tf.keras.activations.relu(x[:, :, :, 2])
    return tf.concat([tanh_activation, tf.expand_dims(scaled_channel_3, axis=-1)], axis=-1)

# Load the model architecture from JSON file
with open('./Model_para/CED_LSTM/SW-Phy/autoencoder_architecture.json', 'r') as json_file:
    loaded_model_json = json_file.read()
autoencoder = model_from_json(loaded_model_json, custom_objects={'custom_activation': custom_activation})

# Load weights into the loaded model
autoencoder.load_weights("./Model_para/CED_LSTM/SW-Phy/autoencoder_weights.h5")

# Extract encoding layers
encode_layers = autoencoder.layers[:6]

# Create a new Sequential model with the extracted layers
encoded = tf.keras.models.Sequential(encode_layers)

# Extract last layers
last_layers = autoencoder.layers[-7:]

# Create a new Sequential model with the extracted last layers
new_model = tf.keras.models.Sequential(last_layers)

encoded_features = encoded.predict(vor_image)
encoded_features_test = encoded.predict(vor_test)
reconstructed_images = autoencoder.predict(vor_image)


In [ ]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, decoder_model, **kwargs):
        super(DecoderLayer, self).__init__(**kwargs)
        # Ensure the decoder_model is a callable (e.g., a Keras model or a function)
        if not callable(decoder_model):
            raise ValueError("decoder_model must be callable")
        self.decoder_model = decoder_model

    def call(self, inputs):
        # It's good practice to check if the inputs are valid for the decoder_model
        # This can be more specific based on the expected input shape, type, etc.
        if inputs is None:
            raise ValueError("Input to DecoderLayer cannot be None")

        # Use decoder_model to decode the inputs
        decoded_images = self.decoder_model(inputs)
        return decoded_images

## LSTM

In [ ]:
datasets = np.split(encoded_features, 30)

# 随机选择数据集用作训练和测试
train_indices, test_indices = train_test_split(np.arange(30), test_size=1/3, shuffle = False)#random_state=42

# 准备训练和测试数据
def prepare_data(indices, datasets, input_length=5, forecast_horizon=5):
    X, y = [], []
    for idx in indices:
        data = datasets[idx]

        for i in range(len(data) - input_length - forecast_horizon + 1):
            X.append(data[i:(i + input_length)])
            y.append(data[(i + input_length):(i + input_length + forecast_horizon)])
    return np.array(X), np.array(y)
x_train, y_train = prepare_data(train_indices, datasets)
x_val, y_val = prepare_data(test_indices, datasets)


# 准备训练和测试数据
def prepare_data(datasets, input_length=5, forecast_horizon=5):
    X, y = [], []
    data = datasets

    for i in range(len(data) - input_length - forecast_horizon + 1):
            X.append(data[i:(i + input_length)])
            y.append(data[(i + input_length):(i + input_length + forecast_horizon)])
    return np.array(X), np.array(y)
x_val, y_val = prepare_data(encoded_features_test)

In [ ]:
class OriginalModel(Model):
    def __init__(self, n_step=5, feature_dim=256):
        super(OriginalModel, self).__init__()
        self.n_step = n_step
        self.feature_dim = feature_dim
        # Define the model structure
        self.lstm1 = LSTM(16, return_sequences=False)
        self.repeatVector = RepeatVector(n_step)
        self.lstm2 = LSTM(16, return_sequences=True)
        self.timeDistributed = TimeDistributed(Dense(feature_dim))
        self.activation = Activation('relu')

    def call(self, inputs):
        x = self.lstm1(inputs)
        x = self.repeatVector(x)
        x = self.lstm2(x)
        x = self.timeDistributed(x)
        return self.activation(x)

    def train_step(self, data):
        x_true, y_true = data  # Unpack the data

        with tf.GradientTape() as tape:
            y_pred = self(x_true, training=True)  # Forward pass
            loss = tf.keras.losses.MeanSquaredError()(y_true, y_pred)  # Compute loss

        gradients = tape.gradient(loss, self.trainable_variables)  # Compute gradients
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))  # Update model parameters

        energy_error = calculate_relative_energy_error(x_true, y_pred)
        mass_loss = calculate_mass_conservation_loss(x_true, y_pred)

        # 更新指标
        self.compiled_metrics.update_state(y_true, y_pred)
        results = {m.name: m.result() for m in self.metrics}
        results.update({"energy_error": energy_error, "mass_loss": mass_loss})
        return results
    def test_step(self, data):
        x_true, y_true = data
        y_pred = self(x_true, training=False)
        loss = tf.keras.losses.MeanSquaredError()(y_true, y_pred)
        # Calculate custom metrics
        energy_error = calculate_relative_energy_error(x_true, y_pred)
        mass_loss = calculate_mass_conservation_loss(x_true, y_pred)

        # 更新指标
        self.compiled_metrics.update_state(y_true, y_pred)
        results = {m.name: m.result() for m in self.metrics}
        results.update({"energy_error": energy_error, "mass_loss": mass_loss})
        return results

In [ ]:
np.random.seed(40)
tf.random.set_seed(40)

Orig_model = OriginalModel(n_step=5, feature_dim=64)

Orig_model.compile(optimizer=tf.keras.optimizers.Adam(0.01), metrics=['mean_squared_error'])#, metrics=['mean_squared_error',calculate_relative_energy_error,calculate_mass_conservation_loss]

Orig_model.fit(x_train, y_train,validation_data = (x_val,y_val),epochs=100, batch_size=64,callbacks = [model_checkpoint])

In [ ]:
np.save('./Dataset_sw/x_train_LSTM.npy', x_train)
np.save('./Dataset_sw/y_train_LSTM.npy', y_train)
np.save('./Dataset_sw/x_val_LSTM.npy', x_val)
np.save('./Dataset_sw/y_val_LSTM.npy', y_val)

In [ ]:
x_train = np.load('./Dataset_sw/x_train_LSTM.npy',mmap_mode = 'r')
y_train = np.load('./Dataset_sw/y_train_LSTM.npy',mmap_mode = 'r')
x_val = np.load('./Dataset_sw/x_val_LSTM.npy',mmap_mode = 'r')
y_val = np.load('./Dataset_sw/y_val_LSTM.npy',mmap_mode = 'r')

In [ ]:
decoder_layer = DecoderLayer(new_model)

x=64
n=5
def preprocess_images(y_true, y_pred):
    decoder_layer = DecoderLayer(new_model)
    y_pred = tf.reshape(y_pred, (-1, x))
    y_true = tf.reshape(y_true, (-1, x))
    y_pred = decoder_layer(y_pred)
    y_true = decoder_layer(y_true)
    y_pred = tf.reshape(y_pred, (-1, n, 64, 64, 3))
    y_true = tf.reshape(y_true, (-1, n, 64, 64, 3))
    return y_true, y_pred

def calculate_image_energy(img):
    g = 9.81
    dx = 1
    img = tf.reshape(img, (-1, img.shape[2], img.shape[3],3))

    kinetic_energy = 0.5 * tf.reduce_sum(tf.square(img[..., 0]), axis=(1, 2)) * dx**2
    kinetic_energy += 0.5 * tf.reduce_sum(tf.square(img[..., 1]), axis=(1, 2)) * dx**2
    # Calculate potential energy
    potential_energy = 0.5 * g * tf.reduce_sum(tf.square(img[..., 2]), axis=(1, 2)) * dx**2
    # Calculate total energy
    total_energy = kinetic_energy + potential_energy
    return total_energy


def calculate_relative_energy_error(y_true, y_pred):
    y_true, y_pred = preprocess_images(y_true, y_pred)
    energy_true = calculate_image_energy(y_true)
    energy_pred = calculate_image_energy(y_pred)
    return tf.reduce_sum(tf.abs(energy_pred - energy_true))
'''
Example Physics Constraints
def calculate_mass_conservation(y):
    y = tf.reshape(y, (-1, y.shape[2], y.shape[3], 3))
    h, u, v = y[..., 2:3], y[..., 0:1], y[..., 1:2]
    dh_dx, dh_dy = tf.image.image_gradients(h)
    du_dx, _ = tf.image.image_gradients(u)
    _, dv_dy = tf.image.image_gradients(v)
    mass_conservation = dh_dx * u + h * du_dx + dh_dy * v + h * dv_dy

    mass_conservation_loss = tf.reduce_sum(tf.abs(mass_conservation), axis=[1, 2, 3])
    return mass_conservation_loss

def calculate_mass_conservation_loss(y_true, y_pred):
    y_true, y_pred = preprocess_images(y_true, y_pred)
    mass_conservation_true = calculate_mass_conservation(y_true)
    mass_conservation_pred = calculate_mass_conservation(y_pred)
    return tf.reduce_mean(tf.abs(mass_conservation_pred - mass_conservation_true))
'''

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)


model_checkpoint = ModelCheckpoint(
    filepath='./Model_para/CED_LSTM/LSTM-Physics-SW',  # Note the removed file extension
    monitor='val_mean_squared_error',  # It's good practice to monitor 'val_loss' for consistency
    save_best_only=True,
    verbose=1,
    mode='min',
    save_format='tf'  # Specify save_format as 'tf'
)


alpha = tf.Variable(5e-10, dtype=tf.float32)


def custom_loss(x_true, y_true, y_pred):
    mse_loss = tf.losses.mean_squared_error(y_true, y_pred)
    energy_restriction = calculate_relative_energy_error(x_true, y_pred)
    return mse_loss + alpha* energy_restriction

class PhysicsInformedLSTM(Model):
    def __init__(self, n_step=10, feature_dim=128):
        super(PhysicsInformedLSTM, self).__init__()
        self.n_step = n_step
        self.feature_dim = feature_dim
        self.lstm1 = LSTM(16, return_sequences=False)
        self.repeatVector = RepeatVector(n_step)
        self.lstm2 = LSTM(16, return_sequences=True)
        self.timeDistributed = TimeDistributed(Dense(feature_dim))
        self.activation = Activation('relu')
        # Define custom metrics
        self.energy_error_metric = tf.keras.metrics.Mean(name='energy_error')

    def call(self, inputs):
        x = self.lstm1(inputs)
        x = self.repeatVector(x)
        x = self.lstm2(x)
        x = self.timeDistributed(x)
        return self.activation(x)


    def train_step(self, data):
        x_true, y_true = data
        with tf.GradientTape() as tape:
            y_pred = self(x_true, training=True)
            loss = custom_loss(x_true, y_true, y_pred)
            # Calculate custom metrics
            energy_error = calculate_relative_energy_error(x_true, y_pred)
        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        # Update built-in and custom metrics
        self.compiled_metrics.update_state(y_true, y_pred)
        self.energy_error_metric.update_state(energy_error)
        results = {m.name: m.result() for m in self.metrics}
        # Include custom metrics in the returned dictionary
        results.update({'energy_error': self.energy_error_metric.result()})
        return results

    def test_step(self, data):
        x_true, y_true = data
        y_pred = self(x_true, training=False)
        loss = custom_loss(x_true, y_true, y_pred)
        # Calculate custom metrics
        energy_error = calculate_relative_energy_error(x_true, y_pred)
        # Update built-in and custom metrics
        self.compiled_metrics.update_state(y_true, y_pred)
        self.energy_error_metric.update_state(energy_error)
        results = {m.name: m.result() for m in self.metrics}
        # Include custom metrics in the returned dictionary
        results.update({'energy_error': self.energy_error_metric.result()})
        return results

    # Reset custom metrics after each epoch
    def reset_metrics(self):
        super(PhysicsInformedLSTM, self).reset_metrics()
        self.energy_error_metric.reset_states()
# Instantiate the PhysicsInformedLSTM model
model = PhysicsInformedLSTM(n_step=5, feature_dim=64)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(), metrics=['mean_squared_error'])
import tensorflow as tf
from tensorflow.keras.callbacks import Callback

# Custom callback to adjust alpha
class AdjustAlphaCallback(Callback):
    def __init__(self, alpha_var, new_value, change_epoch):
        super().__init__()
        self.alpha_var = alpha_var
        self.new_value = new_value
        self.change_epoch = change_epoch

    def on_epoch_end(self, epoch, logs=None):
        if epoch == self.change_epoch:
            tf.keras.backend.set_value(self.alpha_var, self.new_value)
            print(f"Alpha has been updated to {self.new_value} at epoch {epoch+1}")

# Instantiate the callback with the desired settings
adjust_alpha_callback = AdjustAlphaCallback(alpha, new_value=1e-9, change_epoch=50)  # Epochs are 0-indexed, so epoch 29 is the 30th epoch

# Now include this callback in your model's fit method along with other callbacks
model.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=100, batch_size=64, callbacks=[model_checkpoint, adjust_alpha_callback])


In [ ]:

tf.keras.utils.set_random_seed(1)
early_stopping = EarlyStopping(
    monitor='mean_squared_error',
    patience=15,
    verbose=1,
    mode='min',
    restore_best_weights=True
)

# Define the model checkpoint callback
model_checkpoint = ModelCheckpoint(
    filepath='./Model_para/CED_LSTM/LSTM-BP-SW',  # Path where to save the model
    monitor='val_mean_squared_error',
    save_best_only=True, 
    verbose=1,
    mode='min' 
)

reduce_lr = ReduceLROnPlateau(
    monitor='mean_squared_error',
    factor=0.5, 
    patience=10, 
    verbose=1,
    mode='min',  
    min_lr=1e-6  
)

np.random.seed(40)
tf.random.set_seed(40)


Orig_model = OriginalModel(n_step=5, feature_dim=64)

Orig_model.compile(optimizer=tf.keras.optimizers.Adam(0.01), metrics=['mean_squared_error'])#, metrics=['mean_squared_error',calculate_relative_energy_error,calculate_mass_conservation_loss]

Orig_model.fit(x_train, y_train,validation_data = (x_val,y_val),epochs=100, batch_size=64,callbacks = [model_checkpoint])

### Rolling Forecasting

In [ ]:
original_model_path = './Model_para/CED_LSTM/LSTM-BP-SW'
physical_model_path = './Model_para/CED_LSTM/LSTM-Physics-SW'

original_model = tf.keras.models.load_model(original_model_path, custom_objects={
    'calculate_relative_energy_error': calculate_relative_energy_error,
    'calculate_mass_conservation_loss': calculate_mass_conservation_loss
})

physical_model = tf.keras.models.load_model(physical_model_path, custom_objects={
    'custom_loss': custom_loss,
    'calculate_relative_energy_error': calculate_relative_energy_error,
    'calculate_mass_conservation_loss': calculate_mass_conservation_loss
})

In [ ]:
# Load and preprocess data
x_train_cnn, x_test_cnn = train_test_split(encoded_features, test_size=1/3, shuffle=False)
reshaped_features = x_test_cnn.reshape((10, 300, 64))

# Extract features for original model
extracted_features_batches = []
x = 5
for feature_batch in reshaped_features:
    extracted_batch = np.array([feature_batch[i:i+x] for i in range(0, feature_batch.shape[0] - 10, x)])
    extracted_features_batches.append(extracted_batch)
extracted_features_batches = np.array(extracted_features_batches)

# Extract true images for physics-based model
reshaped_features = true_test.reshape((10, 300, 64, 64, 3))
extracted_true_image = []
x = 5
for feature_batch in reshaped_features:
    extracted_batch = np.array([feature_batch[i:i+x] for i in range(10, feature_batch.shape[0], x)])
    extracted_true_image.append(extracted_batch)
extracted_true_image = np.array(extracted_true_image)

In [ ]:
START_POSITION = 15
STEP_SIZE = 5
SEQUENCE_LENGTH = 64

def rolling_forecast(model, initial_input, true_images):
    input_sequence = initial_input.reshape(1, STEP_SIZE, SEQUENCE_LENGTH)  # Reshape to match LSTM input shape
    predictions = []
    mse_array = []
    true_images_array = []
    for i in range(42):  # Consider 5 steps at a time

        next_steps = model.predict(input_sequence)
        next_steps_2d = next_steps.reshape(-1, SEQUENCE_LENGTH)

        predicted_image = decoder_layer(next_steps_2d)
        true_image = true_images[START_POSITION + i]

        predictions.extend(next_steps_2d)
        mse = calculate_rrmse(true_image,predicted_image).reshape(-1, 1)#np.mean((predicted_image - true_image) ** 2, axis=(1, 2, 3)).reshape(-1, 1)

        mse_array.append(mse)
        true_images_array.append(true_image)
        input_sequence = next_steps
    return np.array(predictions), np.array(mse_array), np.array(true_images_array)

def prepare_data(features_batches, true_image_batches, batch_index):
    return features_batches[batch_index], true_image_batches[batch_index]

n = 1
features_batches, true_image_batches = extracted_features_batches[n:(n+9)], extracted_true_image[n:(n+9)]

all_mse_orig = []
all_mse_phy = []
pred_orig_list = []
pred_phy_list = []

true_orig_list = []  # Renamed from t_orig_list
true_phy_list = []  # Renamed from t_phy_list

hist_mse_orig = []
hist_mse_phy = []
# Collect MSE for each data set
for i in range(9):
    x_test_cu, y_test_cu = prepare_data(features_batches, true_image_batches, i)
    initial_input = x_test_cu[START_POSITION, :, :]

    pred_orig, mse_orig, true_orig = rolling_forecast(original_model, initial_input, y_test_cu)
    pred_phy, mse_phy, true_phy = rolling_forecast(physical_model, initial_input, y_test_cu)

    avg_mse_per_group_orig = np.mean(mse_orig, axis=1)
    avg_mse_per_group_phy = np.mean(mse_phy, axis=1)
    hist_mse_orig.append(mse_orig)
    hist_mse_phy.append(mse_phy)

    all_mse_orig.append(avg_mse_per_group_orig)
    all_mse_phy.append(avg_mse_per_group_phy)
    pred_orig = decoder_layer(pred_orig)
    pred_phy = decoder_layer(pred_phy)
    pred_orig_list.append(pred_orig)
    pred_phy_list.append(pred_phy)
    true_orig_list.append(true_orig)
    true_phy_list.append(true_phy)

all_mse_orig = np.array(all_mse_orig)
all_mse_phy = np.array(all_mse_phy)

# Filter out groups with maximum MSE below the threshold
threshold = 1
is_below_threshold_orig = np.max(all_mse_orig, axis=(1, 2)) <= threshold
filtered_mse_orig = all_mse_orig[is_below_threshold_orig]

is_below_threshold_phy = np.max(all_mse_phy, axis=(1, 2)) <= threshold
filtered_mse_phy = all_mse_phy[is_below_threshold_phy]

# Calculate average and standard deviation of filtered MSE
avg_mse_orig = np.mean(filtered_mse_orig, axis=0)
std_mse_orig = np.std(filtered_mse_orig, axis=0)

avg_mse_phy = np.mean(filtered_mse_phy, axis=0)
std_mse_phy = np.std(filtered_mse_phy, axis=0)

# Flatten arrays for plotting
avg_mse_orig = np.asarray(avg_mse_orig).flatten()
std_mse_orig = np.asarray(std_mse_orig).flatten()
avg_mse_phy = np.asarray(avg_mse_phy).flatten()
std_mse_phy = np.asarray(std_mse_phy).flatten()

# Plot aggregated MSE with standard deviation
plt.figure(figsize=(10, 5))
plt.plot(avg_mse_orig, label='Original Model Average MSE')
plt.fill_between(range(len(avg_mse_orig)),
                 avg_mse_orig - std_mse_orig,
                 avg_mse_orig + std_mse_orig,
                 alpha=0.2)
plt.plot(avg_mse_phy, label='Physics-based Model Average MSE')
plt.fill_between(range(len(avg_mse_phy)), avg_mse_phy - std_mse_phy, avg_mse_phy + std_mse_phy, alpha=0.2)

plt.title('Aggregated Images R-RMSE with Standard Deviation over Steps Compared to Decoded True Fields')
plt.xlabel('Step', fontsize=16)
plt.ylabel('MSE', fontsize=16)
plt.legend(loc='upper right', fontsize=12)
plt.grid(True)
plt.show()


In [ ]:
hist_mse_orig = np.array(hist_mse_orig).reshape((-1,1))
hist_mse_phy = np.array(hist_mse_phy).reshape((-1,1))

plt.figure(figsize=(15, 5))

color_orig_hist = 'skyblue'
color_phy_hist = 'orange'
edge_color = 'black'
plt.hist(hist_mse_orig, bins=30, alpha=0.75, color='skyblue', edgecolor='black', label='Basic CED-LSTM Model')
plt.hist(hist_mse_phy, bins=30, alpha=0.75, color='orange', edgecolor='black', label='Physics-based CED-LSTM Model')

plt.xlabel('Error Value', fontsize=14)
plt.ylabel('Frequency', fontsize=14)
plt.title('Histogram of MSE Values for Two Models', fontsize=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(fontsize=12)

plt.grid(axis='y', linestyle='--', alpha=0.6)

plt.show()

In [ ]:
START_POSITION = 15
STEP_SIZE = 5
SEQUENCE_LENGTH = 64

def rolling_forecast(model, initial_input, true_images):
    input_sequence = initial_input.reshape(1, STEP_SIZE, SEQUENCE_LENGTH)  # Reshape to match LSTM input shape
    predictions = []
    ssim_array = []
    true_images_array = []
    for i in range(42):  # Consider 5 steps at a time
        next_steps = model.predict(input_sequence)
        next_steps_2d = next_steps.reshape(-1, SEQUENCE_LENGTH)

        predicted_image = decoder_layer(next_steps_2d)
        true_image = true_images[START_POSITION + i]

        # Convert TensorFlow tensor to NumPy array
        predicted_image_np = predicted_image.numpy()

        predictions.extend(next_steps_2d)
        ssim_val, _ = ssim(predicted_image_np, true_image, multichannel=True, full=True, win_size=5)
        ssim_array.append([ssim_val])
        true_images_array.append(true_image)
        input_sequence = next_steps
    return np.array(predictions), np.array(ssim_array), np.array(true_images_array)

def prepare_data(features_batches, true_image_batches, batch_index):
    return features_batches[batch_index], true_image_batches[batch_index]

n = 1
features_batches, true_image_batches = extracted_features_batches[n:(n+9)], extracted_true_image[n:(n+9)]

all_ssim_orig = []
all_ssim_phy = []
pred_orig_list = []
pred_phy_list = []

true_orig_list = []  # Renamed from t_orig_list
true_phy_list = []  # Renamed from t_phy_list

# Collect SSIM for each data set
for i in range(9):
    x_test_cu, y_test_cu = prepare_data(features_batches, true_image_batches, i)
    initial_input = x_test_cu[START_POSITION, :, :]

    pred_orig, ssim_orig, true_orig = rolling_forecast(original_model, initial_input, y_test_cu)
    pred_phy, ssim_phy, true_phy = rolling_forecast(physical_model, initial_input, y_test_cu)

    avg_ssim_per_group_orig = np.mean(ssim_orig, axis=1)
    avg_ssim_per_group_phy = np.mean(ssim_phy, axis=1)

    all_ssim_orig.append(avg_ssim_per_group_orig)
    all_ssim_phy.append(avg_ssim_per_group_phy)
    pred_orig = decoder_layer(pred_orig)
    pred_phy = decoder_layer(pred_phy)
    pred_orig_list.append(pred_orig)
    pred_phy_list.append(pred_phy)
    true_orig_list.append(true_orig)
    true_phy_list.append(true_phy)

In [ ]:
START_POSITION = 15
STEP_SIZE = 5
SEQUENCE_LENGTH = 64

from skimage.metrics import peak_signal_noise_ratio as psnr

def rolling_forecast(model, initial_input, true_images):
    input_sequence = initial_input.reshape(1, STEP_SIZE, SEQUENCE_LENGTH)  # Reshape to match LSTM input shape
    predictions = []
    psnr_array = []
    true_images_array = []
    for i in range(42):  # Consider 5 steps at a time
        next_steps = model.predict(input_sequence)
        next_steps_2d = next_steps.reshape(-1, SEQUENCE_LENGTH)

        predicted_image = decoder_layer(next_steps_2d)
        true_image = true_images[START_POSITION + i]

        # Convert TensorFlow tensor to NumPy array
        predicted_image_np = predicted_image.numpy()

        predictions.extend(next_steps_2d)
        psnr_val = psnr(true_image, predicted_image_np, data_range=predicted_image_np.max() - predicted_image_np.min())
        psnr_array.append([psnr_val])
        true_images_array.append(true_image)
        input_sequence = next_steps
    return np.array(predictions), np.array(psnr_array), np.array(true_images_array)


def prepare_data(features_batches, true_image_batches, batch_index):
    return features_batches[batch_index], true_image_batches[batch_index]

n = 1
features_batches, true_image_batches = extracted_features_batches[n:(n+9)], extracted_true_image[n:(n+9)]

all_psnr_orig = []
all_psnr_phy = []
pred_orig_list = []
pred_phy_list = []

true_orig_list = []  # Renamed from t_orig_list
true_phy_list = []  # Renamed from t_phy_list

# Collect PSNR for each data set
for i in range(9):
    x_test_cu, y_test_cu = prepare_data(features_batches, true_image_batches, i)
    initial_input = x_test_cu[START_POSITION, :, :]

    pred_orig, psnr_orig, true_orig = rolling_forecast(original_model, initial_input, y_test_cu)
    pred_phy, psnr_phy, true_phy = rolling_forecast(physical_model, initial_input, y_test_cu)

    avg_psnr_per_group_orig = np.mean(psnr_orig, axis=1)
    avg_psnr_per_group_phy = np.mean(psnr_phy, axis=1)

    all_psnr_orig.append(avg_psnr_per_group_orig)
    all_psnr_phy.append(avg_psnr_per_group_phy)
    pred_orig = decoder_layer(pred_orig)
    pred_phy = decoder_layer(pred_phy)
    pred_orig_list.append(pred_orig)
    pred_phy_list.append(pred_phy)
    true_orig_list.append(true_orig)
    true_phy_list.append(true_phy)

all_psnr_orig = np.array(all_psnr_orig)
all_psnr_phy = np.array(all_psnr_phy)

filtered_psnr_orig = all_psnr_orig
filtered_psnr_phy = all_psnr_phy

# Calculate average and standard deviation of filtered PSNR
avg_psnr_orig = np.mean(filtered_psnr_orig, axis=0)
std_psnr_orig = np.std(filtered_psnr_orig, axis=0)

avg_psnr_phy = np.mean(filtered_psnr_phy, axis=0)
std_psnr_phy = np.std(filtered_psnr_phy, axis=0)

# Flatten arrays for plotting
avg_psnr_orig = np.asarray(avg_psnr_orig).flatten()
std_psnr_orig = np.asarray(std_psnr_orig).flatten()
avg_psnr_phy = np.asarray(avg_psnr_phy).flatten()
std_psnr_phy = np.asarray(std_psnr_phy).flatten()

# Plot aggregated PSNR with standard deviation
plt.figure(figsize=(10, 5))
plt.plot(avg_psnr_orig, label='Original Model Average PSNR')
plt.fill_between(range(len(avg_psnr_orig)),
                 avg_psnr_orig - std_psnr_orig,
                 avg_psnr_orig + std_psnr_orig,
                 alpha=0.2)
plt.plot(avg_psnr_phy, label='Physics-based Model Average PSNR')
plt.fill_between(range(len(avg_psnr_phy)), avg_psnr_phy - std_psnr_phy, avg_psnr_phy + std_psnr_phy, alpha=0.2)

plt.title('Aggregated Images PSNR with Standard Deviation over Steps Compared to True Fields')
plt.xlabel('Step', fontsize=16)
plt.ylabel('PSNR', fontsize=16)
plt.legend(loc='upper right', fontsize=12)
plt.grid(True)
plt.show()

In [ ]:
plt.figure(figsize=(15, 5))

# Define color palette
color_orig = 'blue'
color_phy = 'red'
color_fill_orig = 'lightblue'
color_fill_phy = 'salmon'  # Using 'salmon' as a lighter shade of red

# Plot MSE
plt.subplot(1, 3, 1)
plt.plot(avg_mse_orig, label='Basic Model', color=color_orig)
plt.fill_between(range(len(avg_mse_orig)),
                 avg_mse_orig - std_mse_orig,
                 avg_mse_orig + std_mse_orig,
                 alpha=0.2, color=color_fill_orig)
plt.plot(avg_mse_phy, label='Physics-based Model', color=color_phy)
plt.fill_between(range(len(avg_mse_phy)),
                 avg_mse_phy - std_mse_phy,
                 avg_mse_phy + std_mse_phy,
                 alpha=0.2, color=color_fill_phy)
plt.title('R-RMSE with Standard Deviation over Steps', fontsize=16)
plt.xlabel('Step', fontsize=12)
plt.ylabel('R-RMSE', fontsize=12)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(loc='upper left', fontsize=10)
plt.grid(True)

# Plot SSIM
plt.subplot(1, 3, 2)
plt.plot(avg_ssim_orig, label='Basic Model', color=color_orig)
plt.fill_between(range(len(avg_ssim_orig)),
                 avg_ssim_orig - std_ssim_orig,
                 avg_ssim_orig + std_ssim_orig,
                 alpha=0.2, color=color_fill_orig)
plt.plot(avg_ssim_phy, label='Physics-based Model', color=color_phy)
plt.fill_between(range(len(avg_ssim_phy)),
                 avg_ssim_phy - std_ssim_phy,
                 avg_ssim_phy + std_ssim_phy,
                 alpha=0.2, color=color_fill_phy)
plt.title('SSIM with Standard Deviation over Steps', fontsize=16)
plt.xlabel('Step', fontsize=12)
plt.ylabel('SSIM', fontsize=12)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(loc='upper right', fontsize=10)
plt.grid(True)

# Plot PSNR
plt.subplot(1, 3, 3)
plt.plot(avg_psnr_orig, label='Basic Model', color=color_orig)
plt.fill_between(range(len(avg_psnr_orig)),
                 avg_psnr_orig - std_psnr_orig,
                 avg_psnr_orig + std_psnr_orig,
                 alpha=0.2, color=color_fill_orig)
plt.plot(avg_psnr_phy, label='Physics-based Model', color=color_phy)
plt.fill_between(range(len(avg_psnr_phy)),
                 avg_psnr_phy - std_psnr_phy,
                 avg_psnr_phy + std_psnr_phy,
                 alpha=0.2, color=color_fill_phy)
plt.title('PSNR with Standard Deviation over Steps', fontsize=16)
plt.xlabel('Step', fontsize=12)
plt.ylabel('PSNR', fontsize=12)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(loc='upper right', fontsize=10)
plt.grid(True)

plt.tight_layout()
plt.show()